In [3]:
import pandas as pd
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import VarianceThreshold
import numpy as np

In [2]:
df = pd.read_csv("offen.csv")

In [3]:
inputs = df.iloc[:,3:].values
outs = df["labels"].values



selector = SelectKBest(f_classif, k=5)
X_new = selector.fit_transform(inputs, outs)
cols = selector.get_support(indices=True)
selected_patterns = np.take(df.columns.values,[x+3 for x in cols] )
print(selected_patterns)

['[bitch]' '[bitch]|[hoe]' '[bitch]|$LOC' '[hoe]|[bitch]' '$DATE|[bitch]']


In [4]:

df_new = df.select_dtypes(include='number').drop("id", axis = 1)

In [5]:
matrix = df_new.corr().abs()

In [6]:
mask = np.triu(np.ones_like(matrix, dtype=bool))

reduced_matrix = matrix.mask(mask)

In [7]:
reduced_matrix.iloc[:5,:5]

,labels,NUM,NUM|[bitch],NUM|[bitch]+NOUN,NUM|[bitch]+NOUN|AUX
labels,NaN,NaN,NaN,NaN,NaN
NUM,0.117709,NaN,NaN,NaN,NaN
NUM|[bitch],0.445058,0.745356,NaN,NaN,NaN
NUM|[bitch]+NOUN,0.053504,0.222222,0.248452,NaN,NaN
NUM|[bitch]+NOUN|AUX,0.110068,0.362030,0.333333,0.745356,NaN


In [8]:
to_drop = [c for c in reduced_matrix.columns if any(reduced_matrix[c] ==1)]

In [9]:
len(df.columns.values)

900

In [10]:
len(to_drop)

528

In [11]:
df_reduced = df_new.drop(to_drop, axis=1)
df_reduced.shape

(112, 370)

In [12]:
df_reconstructed = df[df_reduced.columns.values]
df_reconstructed = df_reconstructed.drop("labels", axis=1)
# df_reconstructed["id"] = df['id']
# df_reconstructed["sentences"] = df["sentences"]

In [13]:
inputs = df_reconstructed.values
outs = df["labels"].values



selector = SelectKBest(f_classif, k=5)
X_new = selector.fit_transform(inputs, outs)
cols = selector.get_support(indices=True)
selected_patterns = np.take(df_reconstructed.columns.values,[x for x in cols] )
print(selected_patterns)

['[bitch]' '[bitch]|$LOC' '[hoe]|[bitch]' '[get]|[bitch]' '$DATE|[bitch]']


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(16,12))

# Create a custom diverging palette
cmap = sns.diverging_palette(250, 15, s=75, l=40,
                             n=9, center="light", as_cmap=True)

_ = sns.heatmap(matrix, center=0, annot=True, 
                fmt='.2f', square=True, cmap=cmap)

In [115]:
from sklearn.metrics import precision_recall_fscore_support
collector = {}
labels = df['labels']
all_cols = df.columns.values[4:]
for col in all_cols:
    col_selected = df[col]
    fscore = precision_recall_fscore_support(labels, col_selected,  average="binary")[2]
    collector[col] =  fscore

In [116]:
patterns_selected = []

In [117]:
collector = {k: v for k, v in sorted(collector.items(), key=lambda item: item[1])}

In [118]:
starter = list(collector.keys())[-1]


In [119]:
corr = df.corr()

In [120]:
patterns_selected.append(starter)
to_drop = [c for c in corr.columns if corr[starter][c] >= 0.8]
df_reduced = df.drop(to_drop, axis=1)

In [121]:
import numpy as np
df_new = pd.DataFrame()
new_cols = df_reduced.columns.values[4:]
for coll in new_cols:
    
    df_new[coll] = np.logical_or(df_reduced[coll], df[starter])

/tmp/ipykernel_26584/1975192939.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_new[coll] = np.logical_or(df_reduced[coll], df[starter])


In [122]:
df_new.shape

(18, 101)

In [123]:
new_collector = {}
for col in new_cols:
    col_selected = df_reduced[col]
    fscore = precision_recall_fscore_support(labels, col_selected,  average="binary")[2]
    new_collector[col] =  fscore

In [124]:
starter, patterns_selected

('$PERCENT|[pay]', ['$PERCENT|[pay]'])

In [128]:
new_collector = {k: v for k, v in sorted(new_collector.items(), key=lambda item: item[1])}
new_collector
next_starter = list(new_collector.keys())[-1]
patterns_selected.append(next_starter)

In [127]:
next_starter

'[expensive]|[pay]+*+PRON'

In [129]:
corr = df_reduced.corr()
to_drop = [c for c in corr.columns if corr[next_starter][c] >= 0.8]
df_reduced = df_reduced.drop(to_drop, axis=1)

In [131]:
to_drop

['[pay]|[expensive]',
 '[pay]|[expensive]+*+PRON',
 '[pay]|[expensive]+*+PRON+AUX',
 '[expensive]|[pay]',
 '[expensive]|[pay]+*+PRON',
 '[expensive]|[pay]+*+PRON+AUX']

In [194]:
#imports
import numpy as np
from sklearn.metrics import precision_recall_fscore_support


# define df, columns, true labels
def feature_selector(df):

    print(f"==================================Start of Feature Selection===========================================")
    remaining_cols = df.columns.values[4:]
    labels = df['labels']
    patterns_selected = []
    jj = 0
    highest_fscore = "0.0"
    while len(patterns_selected)<10 and len(remaining_cols)>0:
        jj += 1
        print(f"Starting iteration {jj} {len(remaining_cols)}")
        #first calculate the fscore
        collector = {}
        for col in remaining_cols:
            col_selected = df[col]
            fscore = precision_recall_fscore_support(labels, col_selected,  average="binary")[2]
            exists = str(fscore) in collector
            if(exists):
                collector[str(fscore)].append(col)
                
            else:
                collector[str(fscore)] = [col]
        #sort and get a pattern with high fscore
        selected_starter_pattern = list(collector.values())[-1]
        collector = {k: v for k, v in sorted(collector.items(), key=lambda item: item[0])}
        current_fscore = list(collector.keys())[-1]

        if(current_fscore>=highest_fscore):
            highest_fscore = current_fscore
        else:
            # print(f"{highest_fscore} {current_fscore}, {selected_starter_pattern}")
            break
        selected_starter_pattern = list(collector.values())[-1]

        #Group the correlated ones and pick the shortest
        rowss = df[selected_starter_pattern]
        correlation = rowss.corr()
        correlation.loc[:,:] =  np.tril(correlation, k=-1)
        cor = correlation.stack()
        ones = cor[cor ==1].reset_index().loc[:,['level_0','level_1']]
        ones = ones.query('level_0 not in level_1')
        grps = list(ones.groupby('level_0').groups.keys())
        colls = []
        for i in grps:
            groups = ones[ones["level_0"]==i].values
            set_maker = []
            for patterns in groups:
                set_maker += patterns.tolist()
            colls.append(sorted(set_maker, key=len)[0])
            
        for selected_starter_pattern in colls:
            patterns_selected.append(selected_starter_pattern)
            try:
                selected_starter_series = df[selected_starter_pattern][0]
                
                corr = df.corr()
                to_drop = [c for c in corr.columns if corr[selected_starter_pattern][c] >= 0.8] #0.8 chosen at random
                df = df.drop(to_drop, axis=1)

                #create a new df with combination of current one
                remaining_cols = df.columns.values[4:]
                for collumn in remaining_cols:
                    df[collumn] = np.logical_or(df[collumn], selected_starter_series)
            except:
                print("We already removed ", selected_starter_pattern)
            for coll in remaining_cols:
                df[coll] = np.logical_or(df[coll], selected_starter_series)
        
        print(f"Finishing iteration {jj} {len(remaining_cols)}, --- {patterns_selected}, {highest_fscore}")
    
    print(f"---------------------------Summary---------------------------")
    print(f"Patterns {patterns_selected}")
    print(f"Positive examples \n{df[df['labels']==1]['sentences'].values}")
    print(f"Negative examples \n{df[df['labels']==0]['sentences'].values}")

    print(f"==================================End of Feature Selection===========================================")
    return patterns_selected




In [197]:
df = pd.read_csv("offen.csv")

features = feature_selector(df)

==================================Start of Feature Selection===========================================
Starting iteration 1 896
Finishing iteration 1 893, --- ['[hoe]|[bitch]'], 0.8000000000000002
Starting iteration 2 893
---------------------------Summary---------------------------
Patterns ['[hoe]|[bitch]']
Positive examples 
["RT LeezyTheWarrior: Can't be letting them messy hoes mess with yo business..."
 'These hoes soo &#127805;&#127936;' 'b_shiznitt fuck you bitch'
 'Never tell a bitch that I love her'
 'Lmfao but " we don\'t talk " gtfoh &#128514;&#128514; bitch I guessed it! &amp; I was right &#128514;&#128527;'
 'Suck my super monkey balls'
 'BengyBenFrank niggah follow me back &#128529;'
 "RT SortaBad: A boy won the Scripps Spelling Bee by spelling 'knaidel'. Used in a sentence: I'll never get any pussy because I know how to &#8230;"
 "hi_imkadie I wouldn't know cause i don't fuck with these hoes"
 'RT LouieVRee: This bitch feet been through 20 years of slavery '
 'RodMarley

In [184]:
df[df["labels"]==1]['sentences'].values

array(["Hence the food is way too pricey for what's offered\n",
       'I love that for things you order regularly you can establish a standing order and get a 5 % discount ( milk , eggs , etc . ) .',
       "A little expensive but I'll definitely be back time and time again\n",
       'For the price I paid, I get more with a vist to another fast food competitor.',
       "but they are more expensive than what I've paid in the past and what another shop charges\n",
       "For the amenities and $ paid, it's small.\n"], dtype=object)